In [ ]:
# default_exp ppcc

# pico_pi_controller.ppcc
> Command line tool

In [ ]:
# hide
# from nbdev.showdoc import *

In [ ]:
#export
#!/usr/bin/python3

In [ ]:
# export
from sys import byteorder
import threading, _thread
import argparse, logging
import atexit
import datetime, time
import uuid, socket
from CircuitPython_pico_pi_common.codes import *

logger = logging.getLogger()
logging.basicConfig(level = logging.DEBUG)

In [ ]:
# export
tcp_address = ('127.0.0.1', 16164)

def log_txn(fname, message, msg=None):
        """Wrapper for logger."""
        id_str = ID_CODE.decode()
        i2c_str = '|'
        logger.info('%-4s %-7s %-47s %s' % (id_str, fname, message+str(msg or ''), i2c_str))

@atexit.register
def goodbye():
    """Cancel any socket connections"""
    fname='goodbye'
    global ppcc_socket
    if 'ppcc_socket' in locals():
        try:
            ppcc_socket.close()
            log_txn(fname,"TCP socket cleanup completed.")
        except:
            log_txn(fname,"TCP socket error on cleanup.")
    #print(fname,"Exiting.")

def connect_ppdd(command):
    fname='connect_ppdd'
    ppcc_socket = socket.socket()
    ppcc_socket.connect(tcp_address)
    # as this is blocking, a timeout? here or at ppdd?
    # if i2c_addr == 0xFF (all ppds) then multiple replies will be received.

    # Send the command to ppdd
    num_bytes_sent = ppcc_socket.send(bytearray([len(command)]) + command)
    
    cmd_code, i2c_addr, cmdargs, cmd_uid, valid_status = parse_cmd(command)
        
    print("SENT",hex(i2c_addr),CMD_NAME[cmd_code],len(cmdargs),"args:",str(cmdargs)[13:-2],
              "UID",str(hex(int.from_bytes(cmd_uid, byteorder)))[2:]," ", end='' )
    
    # Receive the confirmation echo in reply:
    reply = None
    while reply != command:
        reply = bytearray(ppcc_socket.recv(len(command)))
        if(reply==b''):
            print("Connection closed")
            break
        cmd_code, i2c_addr, cmdargs, cmd_uid, valid_status = parse_cmd(reply)
        print("CONFIRMED")
    ppcc_socket.close()
    
def main():
    fname='main'
    global ppcc_socket
    
    # handle command-line arguments
    parser = argparse.ArgumentParser(description='ppdd argument parser')
    parser.add_argument('command', type=str, help="Command to send")
    parser.add_argument('-c', default='localhost', type=str, help="Default: PPController at localhost")
    parser.add_argument('-a', default='0x13', type=str, help="Default address: bosmang.")
    parser.add_argument('-b', action="store_true", help="Send command to bosmang")
    
    args, unknown = parser.parse_known_args()
    for arg in unknown:
        log_txn(fname,"unrecognized command line arguments: ",arg)
    #if args.command == 'flicker':
    #    print("FLICKER command given")

    cmd_uid = uuid.uuid4().int>>64
    command = bytearray([CMD_CODE['FLICKER']])+ \
              bytearray([0x13])+ \
              bytearray([0x01])+ \
              bytearray(cmd_uid.to_bytes(8,byteorder)) 
    
    # flicker cmd bytes: 1 len, 1 cmd code, 8 uuid, 1 device_address, 1 duration
    # note, the PPC will track the i2c_addr of the sender
    
    # todo: ppcc to ask ppdd on every connect: self addr, bosmang
    
    # setup socket client for messages to & from ppdd
    #for index in range(thread_count):
    connection_thread = threading.Thread(target=connect_ppdd(command))
    #    thread_list.append(connection_thread)
        
    #for index in range(thread_count):
    #    thread_list[index].join()

if __name__ == '__main__':
    try:
        main()
    except KeyboardInterrupt as e:
        # handled by atexit
        #exit(str('Exiting.'))
        pass

NameError: name 'atexit' is not defined

In [ ]:
%tb

No traceback available to show.


In [ ]:
# hide
try:
    from IPython.display import display, Javascript
    display(Javascript('IPython.notebook.save_checkpoint();'))
    from time import sleep
    sleep(0.3)
    from nbdev.export import notebook2script
    notebook2script()
except ModuleNotFoundError:
    pass
    """CircuitPython kernel has no nbdev"""

<IPython.core.display.Javascript object>

Converted 00_core.ipynb.
Converted 50_ppdd.ipynb.
Converted 60_ppcc.ipynb.
Converted index.ipynb.
